In [23]:
import pandas as pd
from pathlib import Path
import re
import warnings
import sys
import os
sys.path.append(os.path.join(sys.path[0], '../src'))

from data_preprocessor import clear_data

warnings.filterwarnings("ignore")

In [48]:
def print_sample(sample):
    """Print one sample from the entire dataset."""

    print(f"Context:{sample['context']}")
    try:
        print(f"Question:{sample['Question']}")
    except KeyError:
        print(f"Question:{sample['question']}")
    try:
        print(f"Answer:{sample['Answer']}")
    except KeyError:
        print(f"Answer:{sample['answer']}")

In [49]:
BASE_DIR = Path('EDA.ipynb').resolve().parent.parent
DATA_DIR = BASE_DIR.joinpath('data')
FILE_NAME = "MedQuAD_Russian.csv"

FILE = DATA_DIR.joinpath(FILE_NAME)

In [50]:
data = pd.read_csv(FILE, sep=",")

print(data.info())
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453 entries, 0 to 1452
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0.1        1453 non-null   int64 
 1   Unnamed: 0          1453 non-null   int64 
 2   Question            1453 non-null   object
 3   URL                 1453 non-null   object
 4   Answer              1453 non-null   object
 5   context             1453 non-null   object
 6   Answer_Start_Index  1453 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 79.6+ KB
None


,Unnamed: 0.1,Unnamed: 0,Question,URL,Answer,context,Answer_Start_Index
0,2,2,Вопрос: Что вызывает синдром Нунан?,https://www.nlm.nih.gov/medlineplus/ency/artic...,Синдром Нунан связан с дефектами нескольких ге...,"Синдром Нунан — это заболевание, присутствующе...",0
1,3,3,Вопрос: Каковы осложнения синдрома Нунан?,https://www.nlm.nih.gov/medlineplus/ency/artic...,- Накопление жидкости в тканях организма (лимф...,"Синдром Нунан — это заболевание, присутствующе...",-1
2,4,4,Вопрос: Как предотвратить синдром Нунан?,https://www.nlm.nih.gov/medlineplus/ency/artic...,"Пары с семейным анамнезом синдрома Нунан, возм...","Синдром Нунан — это заболевание, присутствующе...",2184
3,7,10,Вопрос: Что такое синдром Нунан?,https://www.nlm.nih.gov/medlineplus/ency/artic...,"Синдром Нунан – это заболевание, которое может...","Синдром Нунан — это заболевание, присутствующе...",0
4,8,11,Вопрос: Что вызывает синдром Нунан?,https://www.nlm.nih.gov/medlineplus/ency/artic...,Синдром Нунан связан с дефектами нескольких ге...,"Синдром Нунан — это заболевание, присутствующе...",0
5,12,18,Вопрос: Что такое целиакия – вопросы питания? ...,https://www.nlm.nih.gov/medlineplus/ency/artic...,"Целиакия – это иммунное заболевание, передающе...","Целиакия — это иммунное заболевание, передающе...",0
6,25,32,Вопрос: Что такое целиакия? (Также называется:...,https://www.nlm.nih.gov/medlineplus/celiacdise...,"Целиакия — это иммунное заболевание, при котор...",На этой странице Основные сведения Начните зде...,-1
7,27,39,Вопрос: Что такое чувствительность к глютену? ...,https://www.nlm.nih.gov/medlineplus/glutensens...,"Глютен – это белок, содержащийся в пшенице, рж...","Резюме: Глютен — это белок, содержащийся в пше...",8
8,28,42,Вопрос: Что такое целиакия – вопросы питания? ...,https://www.nlm.nih.gov/medlineplus/ency/artic...,"Целиакия – это иммунное заболевание, передающе...","Целиакия — это иммунное заболевание, передающе...",0
9,31,46,Вопрос: Есть ли у вас информация о Комплексной...,https://www.nlm.nih.gov/medlineplus/ency/artic...,Резюме: Комплексная метаболическая панель пред...,Комплексная метаболическая панель представляет...,-1


In [51]:
print_sample(data.iloc[0])

Context:Синдром Нунан — это заболевание, присутствующее с рождения (врожденное), которое вызывает аномальное развитие многих частей тела. В некоторых случаях он передается по наследству (по наследству). Причины синдрома Нунан связаны с дефектами нескольких генов. В целом, некоторые белки, участвующие в росте и развитии, становятся сверхактивными в результате этих генных изменений. Синдром Нунан является аутосомно-доминантным заболеванием. Это означает, что только один родитель должен передать неработающий ген, чтобы ребенок заболел синдромом. Однако некоторые случаи могут не передаваться по наследству. Симптомы Симптомы включают в себя: Задержку полового созревания. Раскосые или широко посаженные глаза. Потеря слуха (варьируется). Низко посаженные уши или уши неправильной формы. Легкая умственная отсталость (только примерно в 25% случаев). ростМаленький половой членНеопущенные яичкиНеобычная форма груди (чаще всего впалая грудь, называемая воронкообразной грудной клеткой)Перепончатая и

In [52]:
print_sample(data.iloc[10])

Context:Слишком много натрия в вашем рационе может быть вредным для вас. Если у вас высокое кровяное давление или сердечная недостаточность, вас могут попросить ограничить количество соли (содержащей натрий), которую вы едите каждый день. Эти советы помогут вам выбрать продукты с низким содержанием натрия. Соль и ваша диета Для правильной работы вашего организма соль необходима. Соль содержит натрий. Натрий помогает вашему организму контролировать многие функции. Слишком много натрия в вашем рационе может быть вредным для вас. Для большинства людей пищевой натрий поступает из соли, которая содержится в пище или добавляется в нее. Если у вас высокое кровяное давление или сердечная недостаточность, вас, скорее всего, попросят ограничить количество соли, которую вы едите каждый день. Даже люди с нормальным кровяным давлением будут иметь более низкое (и более здоровое) кровяное давление, если они снизят количество потребляемой соли. Диетический натрий измеряется в миллиграммах (мг). Если у

In [53]:
print_sample(data.iloc[15])

Context:Описание Синдром кольцевой хромосомы 20 — это состояние, которое влияет на нормальное развитие и функцию мозга. Наиболее распространенной особенностью этого состояния являются повторяющиеся припадки (эпилепсия) в детстве. Приступы могут возникать днем ​​или ночью во время сна. Их называют парциальными припадками, поскольку они затрагивают только одну область мозга, область, называемую лобной долей. Во многих случаях припадки носят сложный характер и резистентны к лечению противоэпилептическими препаратами. Длительные приступы, известные как бессудорожный эпилептический статус, также характерны для синдрома кольцевой хромосомы 20. Эти эпизоды включают спутанность сознания и изменения в поведении. Большинство людей с синдромом кольцевой хромосомы 20 также имеют некоторую степень умственной отсталости и поведенческих проблем. Хотя эти проблемы могут возникнуть как до, так и после начала эпилепсии, они имеют тенденцию ухудшаться после развития припадков. Дополнительные особенности 

In [54]:
print_sample(data.iloc[100])

Context:Мочевой катетер — это трубка в мочевом пузыре, которая выводит мочу из организма. Эта трубка может оставаться на месте в течение длительного периода времени. В этом случае его называют постоянным катетером. Моча вытекает из мочевого пузыря в мешок за пределами тела. Если у вас установлен постоянный мочевой катетер, у вас больше шансов развить инфекцию мочевыводящих путей (ИМП) в мочевом пузыре или почках. Причины. Многие виды бактерий и грибков могут вызвать катетерную ИМП. Этот тип ИМВП труднее лечить обычными антибиотиками. Распространенными причинами установки постоянного катетера являются: Подтекание мочи (недержание мочи) Невозможность опорожнить мочевой пузырь Операция на мочевом пузыре, простате или влагалище Во время пребывания в больнице у вас может возникнуть Постоянный катетер: Сразу после любого типа операции. Если вы не можете мочиться. Если необходимо контролировать количество выделяемой мочи. Если вы очень больны и не можете контролировать мочеиспускание. Симптом

In [55]:
data["Answer_Start_Index"].value_counts()

-1       450
 0       360
 12       65
 20       55
 8         8
        ... 
 1151      1
 617       1
 159       1
 3129      1
 1346      1
Name: Answer_Start_Index, Length: 391, dtype: int64

### Data Preprocessing
1. Construct the dataset with columns: ["question", "context", "answer_text", "answer_start"]
2. Check all samples with -1 position
3. Remove all occurences 'Вопрос:' from the "Question" column
4. Remove all occurences 'Резюме:' and 'Краткое описание:' from the "Answer" column
5. Remove all samples with context "На этой странице Основные сведения Начать здесь Диагностика и тесты Узнайте больше Сопутствующие вопросы Подробности Смотрите, играйте и учитесь Изображения Статистика исследований и исследования Клинические испытания Журнальные статьи"
6. Remove all '\n, \t, -, —' and extra spaces
7. Lowercase
8. Remove all duplicates: ["question", "context"]

In [56]:
data = data[["Question", "context", "Answer", "Answer_Start_Index"]].rename(columns={"Question":"question", "Answer":"answer", "Answer_Start_Index":"answer_start"})
data

,question,context,answer,answer_start
0,Вопрос: Что вызывает синдром Нунан?,"Синдром Нунан — это заболевание, присутствующе...",Синдром Нунан связан с дефектами нескольких ге...,0
1,Вопрос: Каковы осложнения синдрома Нунан?,"Синдром Нунан — это заболевание, присутствующе...",- Накопление жидкости в тканях организма (лимф...,-1
2,Вопрос: Как предотвратить синдром Нунан?,"Синдром Нунан — это заболевание, присутствующе...","Пары с семейным анамнезом синдрома Нунан, возм...",2184
3,Вопрос: Что такое синдром Нунан?,"Синдром Нунан — это заболевание, присутствующе...","Синдром Нунан – это заболевание, которое может...",0
4,Вопрос: Что вызывает синдром Нунан?,"Синдром Нунан — это заболевание, присутствующе...",Синдром Нунан связан с дефектами нескольких ге...,0
...,...,...,...,...
1448,Вопрос: Что такое дефицит аденозинмонофосфатде...,Описание Дефицит аденозинмонофосфат-дезаминазы...,Дефицит аденозинмонофосфат (АМФ) дезаминазы — ...,12
1449,Вопрос: Сколько людей страдают от дефицита аде...,Описание Дефицит аденозинмонофосфат-дезаминазы...,Дефицит АМФ-дезаминазы является одним из наибо...,174
1450,"Вопрос: Каковы генетические изменения, связанн...",Описание Дефицит аденозинмонофосфат-дезаминазы...,Мутации в гене AMPD1 вызывают дефицит AMP-деза...,1495
1451,Вопрос: Наследуется ли дефицит аденозинмонофос...,Описание Дефицит аденозинмонофосфат-дезаминазы...,Это состояние наследуется по аутосомно-рецесси...,1346


### Check all samples with -1 position

In [57]:
data_tmp = data[data["answer_start"]==-1]
data_tmp

,question,context,answer,answer_start
1,Вопрос: Каковы осложнения синдрома Нунан?,"Синдром Нунан — это заболевание, присутствующе...",- Накопление жидкости в тканях организма (лимф...,-1
6,Вопрос: Что такое целиакия? (Также называется:...,На этой странице Основные сведения Начните зде...,"Целиакия — это иммунное заболевание, при котор...",-1
9,Вопрос: Есть ли у вас информация о Комплексной...,Комплексная метаболическая панель представляет...,Резюме: Комплексная метаболическая панель пред...,-1
10,Вопрос: Есть ли у вас информация о диете с низ...,Слишком много натрия в вашем рационе может быт...,Резюме: Для правильной работы вашему организму...,-1
11,Вопрос: Есть ли у вас информация о 20 здоровых...,Перекусы – это небольшие быстрые мини-блюда. З...,Резюме: Перекусы — это небольшие быстрые мини-...,-1
...,...,...,...,...
1438,Вопрос: Есть ли у вас информация об опасных ма...,"Опасные материалы – это вещества, которые могу...",Краткое описание: Опасные материалы – это веще...,-1
1442,Вопрос: Каковы симптомы гипотермии? (Также наз...,Гипотермия – это опасно низкая температура тел...,"По мере того, как у человека развивается перео...",-1
1444,Вопрос: Что делать при переохлаждении? (Также ...,Гипотермия – это опасно низкая температура тел...,"- НЕ думайте, что человек, найденный неподвижн...",-1
1447,Вопрос: Есть ли у вас информация об измерении ...,Измерение температуры тела может помочь обнару...,Резюме: Измерение температуры тела может помоч...,-1


In [58]:
print_sample(data_tmp.iloc[0])

Context:Синдром Нунан — это заболевание, присутствующее с рождения (врожденное), которое вызывает аномальное развитие многих частей тела. В некоторых случаях он передается по наследству (по наследству). Причины синдрома Нунан связаны с дефектами нескольких генов. В целом, некоторые белки, участвующие в росте и развитии, становятся сверхактивными в результате этих генных изменений. Синдром Нунан является аутосомно-доминантным заболеванием. Это означает, что только один родитель должен передать неработающий ген, чтобы ребенок заболел синдромом. Однако некоторые случаи могут не передаваться по наследству. Симптомы Симптомы включают в себя: Задержку полового созревания. Раскосые или широко посаженные глаза. Потеря слуха (варьируется). Низко посаженные уши или уши неправильной формы. Легкая умственная отсталость (только примерно в 25% случаев). ростМаленький половой членНеопущенные яичкиНеобычная форма груди (чаще всего впалая грудь, называемая воронкообразной грудной клеткой)Перепончатая и

Remove "-" and "—"

In [59]:
print_sample(data_tmp.iloc[2])

Context:Комплексная метаболическая панель представляет собой группу анализов крови. Они дают общую картину химического баланса и метаболизма вашего организма. Метаболизм относится ко всем физическим и химическим процессам в организме, требующим энергии. Как проводится тест Необходим образец крови. Как подготовиться к тесту Не следует есть и пить в течение 8 часов перед тестом. Как будет проходить тест. Когда вводят иглу для взятия крови, некоторые люди чувствуют умеренную боль. Другие чувствуют только укол или покалывание. После этого может возникнуть пульсация или небольшой синяк. Это скоро пройдет. Зачем проводится тест Этот тест дает вашему лечащему врачу информацию о следующем: Как работают ваши почки и печень? Уровни сахара и кальция в крови? Уровни натрия, калия и хлоридов (называемые электролитами) Уровни белка? Ваш врач может назначить этот тест, чтобы проверить вас на наличие побочных эффектов лекарств или диабета, заболеваний печени или почек. Нормальные результаты Нормальные

Such samples will be removed because the Answer is too big

In [60]:
print_sample(data_tmp.loc[1438])

Context:Опасные материалы – это вещества, которые могут нанести вред здоровью человека или окружающей среде. «Опасно» означает «опасно», поэтому с этими материалами необходимо обращаться правильно. Информирование об опасности, или HAZCOM, учит людей работать с опасными материалами и отходами. Существует множество различных видов опасных материалов, в том числе: Химические вещества, например, те, которые используются для очисткаЛекарственные средства, такие как химиотерапия для лечения ракаРадиоактивные материалы, которые используются для рентгеновских лучей или лучевой терапииТкани человека или животных, кровь или другие вещества из организма, которые могут переносить вредные микробыГазы, которые используются для усыпления людей во время операцииОпасные материалы могут причинить вам вред, если они :Прикоснуться к кожеПопасть в глазаПопасть в дыхательные пути или легкие при дыхании Вызвать пожар или взрыв В вашей больнице или на рабочем месте действуют правила обращения с этими материал

In [61]:
print_sample(data_tmp.iloc[400])

Context:Овощи являются важной частью сбалансированной диеты. Многие люди задаются вопросом, так ли полезны для здоровья замороженные и консервированные овощи, как свежие. В целом, овощи, свежие с фермы или только что собранные, полезнее, чем замороженные или консервированные. Но замороженные и консервированные овощи по-прежнему могут быть хорошим выбором. Их необходимо консервировать или замораживать сразу после сбора, пока в них еще сохранились все полезные питательные вещества. Также имейте в виду, сколько соли добавляется в консервированные овощи. Старайтесь покупать овощи без добавления соли и не пережаривайте овощи, свежие, замороженные или консервированные. Вместо того, чтобы варить их в воде в течение длительного времени, их следует слегка приготовить на пару.
Question:Вопрос: Есть ли у вас информация о продуктах питания — свежих, замороженных или консервированных (также называемых: замороженные продукты по сравнению со свежими или консервированными; свежие продукты по сравнению

Remove all occurences "Резюме"

In [62]:
print_sample(data_tmp.iloc[69])

Context:Витилиго — это состояние кожи, при котором происходит потеря цвета (пигмента) на участках кожи. В результате появляются неровные белые пятна, не имеющие пигмента, но кожа выглядит нормальной. Причины Витилиго возникает, когда иммунные клетки разрушают клетки, вырабатывающие коричневый пигмент (меланоциты). Считается, что это разрушение вызвано аутоиммунной проблемой. Аутоиммунное заболевание возникает, когда иммунная система организма, которая обычно защищает организм от инфекции, вместо этого атакует и разрушает здоровые ткани организма. Точная причина витилиго неизвестна. Витилиго может появиться в любом возрасте. В некоторых семьях наблюдается повышенная частота этого заболевания. Витилиго связано с другими аутоиммунными заболеваниями: болезнью Аддисона (расстройство, возникающее, когда надпочечники не производят достаточно гормонов), заболеванием щитовидной железы, пернициозной анемией (снижение количества эритроцитов, возникающее при нарушении функций кишечника). не может 

The answer to is contained in the context

In [63]:
print_sample(data_tmp.iloc[425])

Context:На этой странице Основные сведения Начать здесь Диагностика и тесты Узнайте больше Связанные вопросы Особенности Генетика Смотри, играй и учись Ссылки недоступны Статистика исследований и исследования Клинические испытания Журнальные статьи
Question:Вопрос: Что такое аутоиммунные заболевания?
Answer:Иммунная система вашего организма защищает вас от болезней и инфекций. Но если у вас аутоиммунное заболевание, ваша иммунная система по ошибке атакует здоровые клетки вашего тела. Аутоиммунные заболевания могут поражать многие части тела. Никто точно не знает, что вызывает аутоиммунные заболевания. Они, как правило, бегают семьями. Женщины – особенно афроамериканки, латиноамериканки и коренные американки – имеют более высокий риск развития некоторых аутоиммунных заболеваний. Существует более 80 типов аутоиммунных заболеваний, некоторые из которых имеют схожие симптомы. Из-за этого вашему врачу будет сложно узнать, действительно ли у вас одно из этих заболеваний, и если да, то какое 

There is no meaningful context

## Data preprocessing

In [64]:
# Remove extra words
example = data_tmp.iloc[425]["question"]
print(example.replace('Вопрос:', ''))
example = data_tmp.iloc[400]["answer"]
print(example.replace('Резюме:', ''))
example = data_tmp.iloc[27]["answer"]
print(example.replace('Краткое описание:', ''))

# Remove all samples with insignificant context
ids = []
substr = "На этой странице Основные сведения"
for i in range(data_tmp.shape[0]):
    if substr not in data_tmp.iloc[i]["context"]:
        ids.append(i)
data_tmp = data_tmp.iloc[ids,:]


 Что такое аутоиммунные заболевания?
 Овощи являются важной частью сбалансированной диеты. Многие люди задаются вопросом, так ли полезны для здоровья замороженные и консервированные овощи, как свежие. В целом овощи, свежие с фермы или только что собранные, полезнее, чем замороженные или консервированные. Но замороженные и консервированные овощи по-прежнему могут быть хорошим выбором. Их необходимо консервировать или замораживать сразу после сбора, пока в них еще есть все полезные питательные вещества. Также учитывайте, сколько соли добавляют в консервированные овощи. Старайтесь покупать овощи без добавления соли и не пережаривайте овощи, свежие, замороженные или консервированные. Вместо того, чтобы варить их в воде в течение длительного времени, их следует слегка приготовить на пару.)
 Типичный 12-месячный ребенок демонстрирует определенные физические и умственные способности. Эти навыки называются этапами развития.

Информация: Все дети развиваются немного по-разному. Если вас беспоко

In [65]:
data_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388 entries, 1 to 1452
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   question      388 non-null    object
 1   context       388 non-null    object
 2   answer        388 non-null    object
 3   answer_start  388 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 15.2+ KB


In [66]:
# Remove all '\n, \t, -, —' and extra spaces for all columns
example = data_tmp.iloc[20]["question"]
example = re.sub('[-—\n\t]', '', example)
example = re.sub(' +', ' ', example)
example

'Вопрос: Нужно ли мне обращаться к врачу по поводу ИМП, связанных с катетером? (Также называемые: ИМВП катетерассоциированные; Инфекция мочевыводящих путей катетерассоциированные; Нозокомиальные ИМВП; ИМП, связанные со здравоохранением; Катетерассоциированная бактериурия)'

### Data Preprocessing for the entire dataframe

In [67]:
data = clear_data(data)
data

,question,context,answer,answer_start
0,что вызывает синдром нунан?,"синдром нунан это заболевание, присутствующее ...",синдром нунан связан с дефектами нескольких ге...,0
1,каковы осложнения синдрома нунан?,"синдром нунан это заболевание, присутствующее ...",накопление жидкости в тканях организма (лимфе...,-1
2,как предотвратить синдром нунан?,"синдром нунан это заболевание, присутствующее ...","пары с семейным анамнезом синдрома нунан, возм...",2184
3,что такое синдром нунан?,"синдром нунан это заболевание, присутствующее ...","синдром нунан – это заболевание, которое может...",0
4,что вызывает синдром нунан?,"синдром нунан это заболевание, присутствующее ...",синдром нунан связан с дефектами нескольких ге...,0
...,...,...,...,...
1448,что такое дефицит аденозинмонофосфатдезаминаз...,описание дефицит аденозинмонофосфатдезаминазы ...,дефицит аденозинмонофосфат (амф) дезаминазы эт...,12
1449,сколько людей страдают от дефицита аденозинмо...,описание дефицит аденозинмонофосфатдезаминазы ...,дефицит амфдезаминазы является одним из наибол...,174
1450,"каковы генетические изменения, связанные с де...",описание дефицит аденозинмонофосфатдезаминазы ...,мутации в гене ampd1 вызывают дефицит ampдезам...,1495
1451,наследуется ли дефицит аденозинмонофосфатдеза...,описание дефицит аденозинмонофосфатдезаминазы ...,это состояние наследуется по аутосомнорецессив...,1346


In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1356 entries, 0 to 1452
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   question      1356 non-null   object
 1   context       1356 non-null   object
 2   answer        1356 non-null   object
 3   answer_start  1356 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 53.0+ KB


In [70]:
data.groupby(["question", "context"])["answer"].count()

question                                                                                                                                                                       context                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [71]:
# Remove duplicates
data.drop_duplicates(subset=['question', 'context'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997 entries, 0 to 1452
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   question      997 non-null    object
 1   context       997 non-null    object
 2   answer        997 non-null    object
 3   answer_start  997 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 38.9+ KB


### Save Dataset

In [72]:
DATA_NAME = "MedQuAD_Rus_clean.csv"
FILE = DATA_DIR.joinpath(DATA_NAME)

data.to_csv(FILE)